##### Labeling using PyTorch model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import os,glob
import cv2
import random
from termcolor import colored

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define MNIST PyTorch MODEL (cut and pase it from MNIST model)-------------------
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        # nn.init.xavier_normal_(self.conv1.weight)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3)
        # nn.init.xavier_normal_(self.conv2.weight)
        self.drop1 = nn.Dropout2d(0.3)
        self.drop2 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(800, 64)
        # nn.init.xavier_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(64, 302)
        # nn.init.xavier_normal_(self.fc2.weight)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)) , 2)
        x= self.drop1(x)
        x= self.flatten(x)
        x= self.fc1(x)
        x= self.drop2(x)
        x= self.fc2(x)
        # return F.log_softmax(x, dim=1)
        return x

network = Net() # model initialization 
# load saved model 
network.load_state_dict(torch.load("/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/chkpoint/July27_acc_0.9994_iter_265_model.pth"))
# network.load_state_dict(torch.load("/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/chkpoint/July27_acc_0.9994_iter_265_model.pt"))

network.eval() # use for evaluation to remove dropout layers , ....


Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (drop1): Dropout2d(p=0.3, inplace=False)
  (drop2): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=800, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=302, bias=True)
)

In [3]:
#Load the test data for validation purpose
TestRoot = "/Users/kpangalu/Downloads/Pytorch_Mnist_28Jul/0720_falsedata/0720_false"

all_imgs = glob.glob(os.path.join(TestRoot, "*.png"))

for fimg in all_imgs:
    fnme1 = fimg.split("\\")[-1]
    fnme2 = fnme1.split('.')[-2]
    #print(fnme2)
    
    img = cv2.imread(fimg,0)
    
    eps = 0.00001
    img = np.float32(img)+eps
    img = img/np.max(img)
    img = img.reshape([1,1,14,24]) # pytorch and tflow [1,14,24,1]
    
    #predictions
    input_torch = torch.from_numpy(img)
    test_pred = network(input_torch)
    test_pred_class_label = torch.argmax(test_pred, dim=1).item()
    prdnum = np.int(test_pred_class_label)
    
    if(prdnum < 10):
        numm = '00' + str(prdnum)
    elif(prdnum > 9 and prdnum <= 99):
        numm = '0' + str(prdnum)
    elif(prdnum > 99):
        numm = str(prdnum)
        
    ren_fle = TestRoot + '/' + str(numm) +'_' + fnme2 + '.png'
    os.rename(fimg, ren_fle)
    
    
    
